# 2020 Presidential Election Time Series
## Project Setup

In [1]:
%run get_data.py

In [ ]:
update_notebook()

In [ ]:
# https://www.codegrepper.com/code-examples/python/run+a+function+every+hour+python
# from time import time, sleep
# from IPython.display import clear_output
# print('begin')
# while True:
#     clear_output()
#     sleep(1 - time() % 1)
    
#     print('clear')
# 	# thing to run

In [ ]:
# def countdown(t):
    
#     print('Updating data')
#     %run get_data.py
    
#     while t: 
#         mins, secs = divmod(t, 60) 
#         timer = '{:02d}:{:02d}'.format(mins, secs) 
#         print(timer, end="\r")
#         time.sleep(1) 
#         t -= 1
    
#     countdown(int(t))
    
# # input time in seconds 
# t = 3600 # input("Enter the time in seconds: ") 
  
# # function call 
# countdown(int(t)) 